#### Script per extreure metadades de múltiples datasets
# OBSERVACIÓ:
Si teniu dubtes sobre el codi, poseu-vos en contacte amb rdr-contacte@csuc.cat
## OBJECTIU DE L'SCRIPT
L'objectiu principal d'aquest script és extreure metadades dels datasets d'una o diverses institucions.





In [1]:
# @title Instal·leu o actualitzeu les llibreries (feu clic al botó Executar &#x25B6; )
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def install_packages(b):
    """
    Function to install or update required Python packages.

    Args:
    b (widget): Button widget that triggers the installation process.

    Returns:
    None
    """
    clear_output(wait=True)
    !pip install --upgrade pip -q  # Upgrade pip silently
    !pip install pyDataverse -q    # Install or update pyDataverse silently
    !pip install requests -q       # Install or update requests silently
    print("Les llibreries s'han descarregat o actualitzat correctament.")

# Displaying installation message
display(HTML("<p style='font-size:14px;'><b>Feu clic al botó següent per instal·lar les llibreries necessàries.</b></p>"))

# Creating installation button
install_button = widgets.Button(description='Instal·lar llibreries')
install_button.on_click(install_packages)

# Displaying the installation button
display(install_button)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00
Les llibreries s'han descarregat o actualitzat correctament.


In [3]:
# @title Feu clic al botó Executar &#x25B6; , ompliu el token i trieu les institucions i les metadades
import subprocess
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from pyDataverse.api import NativeApi, DataAccessApi, MetricsApi
from pyDataverse.models import Dataverse
import pandas as pd
import requests
import logging
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
class UtilsConnection:
    def __init__(self, config):
        self.config = config

    def call_api(self, url, method, data=None):
        payload = {}
        headers = {'X-Dataverse-key': config.get_token()}

        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()

class Config:
    def __init__(self, api_url, logger, token):
        self.api_url = api_url
        self.logger = logger
        self.token = token

    def get_app_config(self):
        return self

    def get_api_url(self):
        return self.api_url

    def get_logger(self):
        return self.logger

    def get_token(self):
        return self.token

class DatasetProcessor:
    def __init__(self, config, identifier):
        self.config = config
        self.list_datasets = []
        self.list_dataverse_children = []
        self.identifier = identifier

    def update_list_dataset(self, dataset_id):
        self.list_datasets.append(dataset_id)

    def update_list_dataverse_children(self, dataseverse_id):
        self.list_dataverse_children.append(dataseverse_id)

    def remove_id_list_dataverse_children(self, dataseverse_id):
        self.list_dataverse_children.remove(dataseverse_id)

    def get_list_datasets(self):
        return self.list_datasets

    def get_list_dataverse_children(self):
        return self.list_dataverse_children

    def count(self):
        return len(self.list_datasets)

    def create_list_datasets(self, identifier):

        conn = UtilsConnection(self.config)

        url_api = f"{self.config.get_api_url()}/api/dataverses/{identifier}/contents"
        object_json = conn.call_api(url_api, "GET")

        if object_json:
            self.config.get_logger().info(f"Reading the API values")
            array_json = object_json.get("data", {})

            for value in array_json:
                if value['type'] == 'dataverse':
                    self.update_list_dataverse_children(value['id'])
                elif value['type'] == 'dataset' and value['protocol'] == 'doi':
                    self.update_list_dataset(value['protocol'] + ':' + value['authority'] + '/' + value['identifier'])
        else:
            self.config.get_logger().error(f"Call API ERROR")

        if not identifier == self.identifier:
            self.remove_id_list_dataverse_children(identifier)

        if len(self.get_list_dataverse_children()) != 0:

            self.create_list_datasets(self.get_list_dataverse_children()[0])
def extract_value(data_dict):
    """
    Function to extract type names and values from a JSON metadata dictionary.

    Args:
    data_dict (dict): JSON metadata dictionary.

    Returns:
    tuple: Type names and values extracted from the metadata dictionary.
    """
    if isinstance(data_dict, dict):
        type_names = []
        values = []
        for key, value in data_dict.items():
            if key == 'typeName' and 'value' in data_dict:
                if isinstance(data_dict['value'], list):
                    for v in data_dict['value']:
                        type_names.append(data_dict['typeName'])
                        values.append(v)
                else:
                    type_names.append(data_dict['typeName'])
                    values.append(data_dict['value'])
            elif isinstance(value, dict) and 'typeName' in value and 'value' in value:
                type_names.append(value['typeName'])
                values.append(value['value'])
            elif isinstance(value, str) and key == 'typeName':
                type_names.append(value)
                values.append(value)
            else:
                extracted_type_names, extracted_values = extract_value(value)
                type_names += extracted_type_names
                values += extracted_values
        return type_names, values
    elif isinstance(data_dict, list):
        type_names = []
        values = []
        for item in data_dict:
            extracted_type_names, extracted_values = extract_value(item)
            type_names += extracted_type_names
            values += extracted_values
        return type_names, values
    else:
        return [], []

def export_metadata(base_url, token, doi, data_keys, data_values, citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values, customUAB_keys, customUAB_values, stateDataset):
    """
    Function to export metadata from a dataset and store it in respective lists.

    Args:
    base_url (str): Base URL of the Dataverse repository.
    token (str): API token for authentication.
    doi (str): DOI of the dataset.
    citation_keys (list): List to store citation metadata keys.
    citation_values (list): List to store citation metadata values.
    geo_keys (list): List to store geospatial metadata keys.
    geo_values (list): List to store geospatial metadata values.
    social_keys (list): List to store social science metadata keys.
    social_values (list): List to store social science metadata values.
    astronomy_keys (list): List to store astronomy metadata keys.
    astronomy_values (list): List to store astronomy metadata values.
    biomedical_keys (list): List to store biomedical metadata keys.
    biomedical_values (list): List to store biomedical metadata values.
    journal_keys (list): List to store journal metadata keys.
    journal_values (list): List to store journal metadata values.
    customUAB_keys (list): List to store customUAB metadata keys.
    customUAB_values (list): List to store customUAB metadata values.

    Returns:
    None
    """
    from pyDataverse.api import NativeApi, DataAccessApi
    from pyDataverse.models import Dataverse
    api = NativeApi(base_url, token)  # Function to access the API
    data_api = DataAccessApi(base_url, token)  # Function to access data via the API
    try:
        dataset = api.get_dataset(doi)  # Retrieve dataset metadata

      # Extract dataset dates metadata if available
        dataset_data = dataset.json()['data']['latestVersion']  # Extract once

        date_keys_all = [
            "publicationDate", "lastUpdateTime", "releaseTime",
            "createTime", "productionDate", "citationDate"
        ]

        for key in date_keys_all:
            if key in dataset_data:
                data_keys.append(key)
                data_values.append(dataset_data[key])

      # Extract citation metadata if available
        if 'citation' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_citation = dataset.json()['data']['latestVersion']['metadataBlocks']['citation']['fields']
            citation = extract_value(metadata_citation)
            citation_keys.extend(citation[0])
            citation_values.extend(citation[1])
            for item in metadata_citation:
                if isinstance(item['value'], str):
                    index_change = citation_keys.index(item['typeName'])
                    citation_values[index_change] = item['value']

        # Extract geospatial metadata if available
        if 'geospatial' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_geospatial = dataset.json()['data']['latestVersion']['metadataBlocks']['geospatial']['fields']
            geospatial = extract_value(metadata_geospatial)
            geo_keys.extend(geospatial[0])
            geo_values.extend(geospatial[1])
            for item in metadata_geospatial:
                if isinstance(item['value'], str):
                    index_canvi = geo_keys.index(item['typeName'])
                    geo_values[index_canvi] = item['value']

        # Extract social science metadata if available
        if 'socialscience' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_socialscience = dataset.json()['data']['latestVersion']['metadataBlocks']['socialscience']['fields']
            socialscience = extract_value(metadata_socialscience)
            social_keys.extend(socialscience[0])
            social_values.extend(socialscience[1])
            for item in metadata_socialscience:
                if isinstance(item['value'], str):
                    index_canvi = social_keys.index(item['typeName'])
                    social_values[index_canvi] = item['value']

        # Extract astronomy metadata if available
        if 'astrophysics' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_astronomy = dataset.json()['data']['latestVersion']['metadataBlocks']['astrophysics']['fields']
            astronomy = extract_value(metadata_astronomy)
            astronomy_keys.extend(astronomy[0])
            astronomy_values.extend(astronomy[1])
            for item in metadata_astronomy:
                if isinstance(item['value'], str):
                    index_canvi = astronomy_keys.index(item['typeName'])
                    astronomy_values[index_canvi] = item['value']

        # Extract biomedical metadata if available
        if 'biomedical' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_biomedical = dataset.json()['data']['latestVersion']['metadataBlocks']['biomedical']['fields']
            biomedical = extract_value(metadata_biomedical)
            biomedical_keys.extend(biomedical[0])
            biomedical_values.extend(biomedical[1])
            for item in metadata_biomedical:
                if isinstance(item['value'], str):
                    index_canvi = biomedical_keys.index(item['typeName'])
                    biomedical_values[index_canvi] = item['value']

        # Extract journal metadata if available
        if 'journal' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_journal = dataset.json()['data']['latestVersion']['metadataBlocks']['journal']['fields']
            journal = extract_value(metadata_journal)
            journal_keys.extend(journal[0])
            journal_values.extend(journal[1])
            for item in metadata_journal:
                if isinstance(item['value'], str):
                    index_canvi = journal_keys.index(item['typeName'])
                    journal_values[index_canvi] = item['value']

        # Extract Library UAB metadata if available

        if 'customUAB' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_customUAB = dataset.json()['data']['latestVersion']['metadataBlocks']['customUAB']['fields']
            customUAB = extract_value(metadata_customUAB)
            customUAB_keys.extend(customUAB[0])
            customUAB_values.extend(customUAB[1])
            for item in metadata_customUAB:
                if isinstance(item['value'], str):
                    index_change = customUAB_keys.index(item['typeName'])
                    customUAB_values[index_change] = item['value']

    except KeyError or InvalidSchema:
        print('S\'ha produït un error en llegir les metadades del conjunt de dades: ' + doi)

def extract_metadata(data, data_keys,data_values,citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values, customUAB_keys, customUAB_values):
    for key, value in zip(data_keys, data_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(citation_keys, citation_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(geo_keys, geo_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(social_keys, social_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(astronomy_keys, astronomy_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(biomedical_keys, biomedical_values):
        if not isinstance(value, dict):
            data.append([key, value])

    for key, value in zip(journal_keys, journal_values):
        if not isinstance(value, dict):
            data.append([key, value])
    for key, value in zip(customUAB_keys, customUAB_values):
        if not isinstance(value, dict):
            data.append([key, value])

# Configuration and execution
import logging
import ipywidgets as widgets
from IPython.display import display

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ask the user for the token
token = input("Introduïu el vostre token i premeu enter: ")

# Use the token in your configuration
config = Config(api_url="https://dataverse.csuc.cat/", logger=logger, token=token)

# List of institutions
institucions = [
    'UB', 'UAB', 'UPC', 'UPF', 'UdG', 'UdL', 'URV', 'UOC', 'UVIC-UCC',
    'URL', 'UIC', 'UIB', 'Agrotecnio', 'CED', 'CRAG', 'CREAF', 'CRM', 'CTFC',
    'i2CAT', 'I3PT', 'IBEC', 'IBEI', 'ICAC-CERCA', 'ICFO-CERCA', 'ICN2',
    'ICRA-CERCA', 'IDIBAPS', 'IDIBELL', 'IDIBGI-CERCA', 'IFAE', 'IJC','IRSantPau','CVC','IRSJD',
    'IPHES-CERCA', 'IRBBarcelona-CERCA', 'IRB', 'IRSICAIXA', 'IRTA',
    'ISGLOBAL', 'VHIR'
]

# Define metadata categories
data_metadata = {"publicationDate":"Publication Date",
                 "createTime":"Create Time",
                 "lastUpdateTime":"Last Update Time",
                 "releaseTime": "Release Time",
                 "productionDate": "Production Date",
                 "citationDate": "Citation Date" }
citation_metadata = {
    "PreviousDatasetPersistentID": "Previous Dataset Persistent ID",
    "title": "Title",
    "subtitle": "Subtitle",
    "alternativeTitle": "Alternative Title",
    "alternativeURL": "Alternative URL",
    "otherIdAgency": "Other ID Agency",
    "otherIdValue": "Other ID Value",
    "authorName": "Author Name",
    "authorAffiliation": "Author Affiliation",
    "authorIdentifierScheme": "Author Identifier Scheme",
    "authorIdentifier": "Author Identifier",
    "datasetContactName": "Dataset Contact Name",
    "datasetContactAffiliation": "Dataset Contact Affiliation",
    "datasetContactEmail": "Dataset Contact Email",
    "dsDescriptionValue": "Description Value",
    "dsDescriptionDate": "Description Date",
    "subject": "Subject",
    "keywordValue": "Keyword",
    "keywordVocabulary": "Keyword Vocabulary",
    "keywordVocabularyURI": "Keyword Vocabulary URI",
    "topicClassValue": "Topic Class",
    "topicClassVocab": "Topic Class Vocabulary",
    "topicClassVocabURI": "Topic Class Vocabulary URI",
    "publicationRelationType": "Publication Relation Type",
    "publicationCitation": "Publication Citation",
    "publicationIDType": "Publication ID Type",
    "publicationIDNumber": "Publication ID Number",
    "publicationURL": "Publication URL",
    "notesText": "Notes",
    "language": "Language",
    "producerName": "Producer Name",
    "producerAffiliation": "Producer Affiliation",
    "producerAbbreviation": "Producer Abbreviation",
    "producerURL": "Producer URL",
    "productionDate": "Production Date",
    "productionPlace": "Production Place",
    "contributorType": "Contributor Type",
    "contributorName": "Contributor Name",
    "grantNumberAgency": "Grant Number Agency",
    "grantNumberValue": "Grant Number Value",
    "distributorAffiliation": "Distributor Affiliation",
    "distributorAbbreviation": "Distributor Abbreviation",
    "distributorURL": "Distributor URL",
    "distributionDate": "Distribution Date",
    "depositor": "Depositor",
    "dateOfDeposit": "Date of Deposit",
    "timePeriodCoveredStart": "Time Period Covered Start",
    "timePeriodCoveredEnd": "Time Period Covered End",
    "dateOfCollectionStart": "Date of Collection Start",
    "dateOfCollectionEnd": "Date of Collection End",
    "kindOfData": "Kind of Data",
    "seriesName": "Series Name",
    "seriesInformation": "Series Information",
    "softwareName": "Software Name",
    "softwareVersion": "Software Version",
    "relatedMaterial": "Related Material",
    "relatedDatasets": "Related Datasets",
    "otherReferences": "Other References",
    "dataSources": "Data Sources",
    "originOfSources": "Origin of Sources",
    "characteristicOfSources": "Characteristic of Sources",
    "accessToSources": "Access to Sources"
}


geospatial_metadata = {
    "country": "Country",
    "state": "State",
    "city": "City",
    "otherGeographicCoverage": "Other Geographic Coverage",
    "geographicUnit": "Geographic Unit",
    "westLongitude": "West Longitude",
    "eastLongitude": "East Longitude",
    "northLongitude": "North Longitude",
    "southLongitude": "South Longitude"
}


social_metadata = {
    "unitOfAnalysis": "Unit of Analysis",
    "universe": "Universe",
    "timeMethod": "Time Method",
    "dataCollector": "Data Collector",
    "collectorTraining": "Collector Training",
    "frequencyOfDataCollection": "Frequency of Data Collection",
    "samplingProcedure": "Sampling Procedure",
    "targetSampleActualSize": "Target Sample Actual Size",
    "targetSampleSizeFormula": "Target Sample Size Formula",
    "deviationsFromSampleDesign": "Deviations From Sample Design",
    "collectionMode": "Collection Mode",
    "researchInstrument": "Research Instrument",
    "dataCollectionSituation": "Data Collection Situation",
    "actionsToMinimizeLoss": "Actions to Minimize Loss",
    "controlOperations": "Control Operations",
    "weighting": "Weighting",
    "cleaningOperations": "Cleaning Operations",
    "datasetLevelErrorNotes": "Dataset Level Error Notes",
    "responseRate": "Response Rate",
    "samplingErrorEstimates": "Sampling Error Estimates",
    "otherDataAppraisal": "Other Data Appraisal",
    "socialScienceNotesType": "Social Science Notes Type",
    "socialScienceNotesSubject": "Social Science Notes Subject",
    "socialScienceNotesText": "Social Science Notes Text"
}


astronomy_metadata = {
    "astroType": "Astro Type",
    "astroFacility": "Astro Facility",
    "astroInstrument": "Astro Instrument",
    "astroObject": "Astro Object",
    "resolution.Spatial": "Spatial Resolution",
    "resolution.Spectral": "Spectral Resolution",
    "resolution.Temporal": "Temporal Resolution",
    "coverage.Spectral.Bandpass": "Spectral Bandpass",
    "coverage.Spectral.CentralWavelength": "Central Wavelength",
    "coverage.Spectral.MinimumWavelength": "Minimum Wavelength",
    "coverage.Spectral.MaximumWavelength": "Maximum Wavelength",
    "coverage.Temporal.StartTime": "Temporal Start Time",
    "coverage.Temporal.StopTime": "Temporal Stop Time",
    "coverage.Spatial": "Spatial Coverage",
    "coverage.Depth": "Depth",
    "coverage.ObjectDensity": "Object Density",
    "coverage.ObjectCount": "Object Count",
    "coverage.SkyFraction": "Sky Fraction",
    "coverage.Polarization": "Polarization",
    "redshiftType": "Redshift Type",
    "resolution.Redshift": "Redshift Resolution",
    "coverage.Redshift.MinimumValue": "Minimum Redshift",
    "coverage.Redshift.MaximumValue": "Maximum Redshift"
}


lifesciences_metadata = {
    "studyDesignType": "Study Design Type",
    "studyFactorType": "Study Factor Type",
    "studyAssayOrganism": "Assay Organism",
    "studyAssayOtherOrganism": "Other Assay Organism",
    "studyAssayMeasurementType": "Assay Measurement Type",
    "studyAssayOtherMeasurementType": "Other Assay Measurement Type",
    "studyAssayTechnologyType": "Assay Technology Type",
    "studyAssayPlatform": "Assay Platform",
    "studyAssayCellType": "Assay Cell Type"
}

journal_metadata = {
    "journalVolume": "Journal Volume",
    "journalIssue": "Journal Issue",
    "journalPubDate": "Journal Publication Date",
    "journalArticleType": "Journal Article Type"
}
customUAB_metadata={"reviewLibrary":"Library reviewing and publishing"}

# Add an option for selecting all
options_institucions = ['Totes les institucions'] + institucions

# Create widgets for instructions and selections
instruction_text_institucions = widgets.HTML(
    value="<b>Trieu una o més institucions:</b>"
)

# Create the widget for multiple selection of institutions
institucions_widget = widgets.SelectMultiple(
    options=options_institucions,
    value=[],
    description='Institucions:',
    disabled=False
)
# Function to save selected institutions and print them
opcions = set()
def save_selection_institucions(change):
    global opcions
    selected = set(change['new'])

    if 'Totes les institucions' in selected:
        opcions = set(institucions)  # Select all institutions if 'Totes les institucions' is chosen
    else:
        opcions.update(selected)  # Update the set with new selections

    print(f"Institucions: {list(opcions)}")

# Function to clear the selection and reset the institutions widget
def restart_selection_institucions(button):
    global opcions
    institucions_widget.value = []  # Clear the selections in the widget
    opcions.clear()  # Clear the global opcions set
    print("La selecció de la institució s'ha restablert.")

# Observe changes in the widget selection for institutions
institucions_widget.observe(save_selection_institucions, names='value')

# Create a button to restart the selection for institutions
restart_button_institucions = widgets.Button(description="Reiniciar la selecció d'institucions")
restart_button_institucions.on_click(restart_selection_institucions)

# Display the widgets and button with instructions

display(instruction_text_institucions, institucions_widget, restart_button_institucions)

###################################################
# DATA METADATA

# Add an option for selecting all
options_data = ['Totes les metadades'] + list(data_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada de dates:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_data,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global opcions set for institutions and selected metadata set
selected_data = set()

# Function to save the selected options for metadata
def save_selection_metadata(change):
    global selected_data
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_data = set(list(data_metadata.keys()))  # Select all metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_data.update(
            key for key, value in data_metadata.items() if value in selected
        )

    print(f"Metadades de dades seleccionades: {list(selected_data)}")

# Function to clear the selection and reset the metadata widget
def restart_selection_metadata(button):
    global selected_data
    metadata_widget.value = []  # Clear the selections in the widget
    selected_data.clear()  # Clear the global selected set
    print("La selecció de metadades de dades s'ha restablert.")

# Observe changes in the widget selection for metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades de dades")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# CITATION METADATA

# Add an option for selecting all
options_citation = ['Totes les metadades'] + list(citation_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada de citació:</b>"
)


# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_citation,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global opcions set for institutions and selected metadata set
selected_citation = set()

# Function to save the selected options for metadata
def save_selection_metadata(change):
    global selected_citation
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_citation = set(list(citation_metadata.keys()))  # Select all metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_citation.update(
            key for key, value in citation_metadata.items() if value in selected
        )

    print(f"Metadades de citació seleccionades: {list(selected_citation)}")

# Function to clear the selection and reset the metadata widget
def restart_selection_metadata(button):
    global selected_citation
    metadata_widget.value = []  # Clear the selections in the widget
    selected_citation.clear()  # Clear the global selected set
    print("La selecció de metadades de citació s'ha restablert.")

# Observe changes in the widget selection for metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades de citació")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)


###########
# GEOSPATIAL METADATA

# Add an option for selecting all geospatial metadata
options_geospatial = ['Totes les metadades'] + list(geospatial_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada geoespacial:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_geospatial,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected geospatial metadata
selected_geospatial = set()

# Function to save the selected options for geospatial metadata
def save_selection_metadata(change):
    global selected_geospatial
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_geospatial = set(list(geospatial_metadata.keys()))  # Select all geospatial metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_geospatial.update(
            key for key, value in geospatial_metadata.items() if value in selected
        )

    print(f"Metadades geospacials seleccionades: {list(selected_geospatial)}")

# Function to clear the selection and reset the geospatial metadata widget
def restart_selection_metadata(button):
    global selected_geospatial
    metadata_widget.value = []  # Clear the selections in the widget
    selected_geospatial.clear()  # Clear the global selected set
    print("La selecció de metadades geospacials s'ha restablert.")

# Observe changes in the widget selection for geospatial metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for geospatial metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades geospacials")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# SOCIAL METADATA

# Add an option for selecting all social metadata
options_social = ['Totes les metadades'] + list(social_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada social:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_social,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected social metadata
selected_social = set()

# Function to save the selected options for social metadata
def save_selection_metadata(change):
    global selected_social
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_social = set(list(social_metadata.values()))  # Select all social metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_social.update(
            key for key, value in social_metadata.items() if value in selected
        )

    print(f"Metadades socials seleccionades: {list(selected_social)}")

# Function to clear the selection and reset the social metadata widget
def restart_selection_metadata(button):
    global selected_social
    metadata_widget.value = []  # Clear the selections in the widget
    selected_social.clear()  # Clear the global selected set
    print("La selecció de metadades socials s'ha restablert.")

# Observe changes in the widget selection for social metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for social metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades socials")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# ASTRONOMY METADATA

# Add an option for selecting all astronomy metadata
options_astronomy = ['Totes les metadades'] + list(astronomy_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada d'astronomia:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_astronomy,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected astronomy metadata
selected_astronomy = set()

# Function to save the selected options for astronomy metadata
def save_selection_metadata(change):
    global selected_astronomy
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_astronomy = set(list(astronomy_metadata.keys()))  # Select all astronomy metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_astronomy.update(
            key for key, value in astronomy_metadata.items() if value in selected
        )


    print(f"Metadades d'astronomia seleccionades: {list(selected_astronomy)}")

# Function to clear the selection and reset the astronomy metadata widget
def restart_selection_metadata(button):
    global selected_astronomy
    metadata_widget.value = []  # Clear the selections in the widget
    selected_astronomy.clear()  # Clear the global selected set
    print("La selecció de metadades d'astronomia s'ha restablert.")

# Observe changes in the widget selection for astronomy metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for astronomy metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades d'astronomia")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# LIFE SCIENCES METADATA

# Add an option for selecting all life sciences metadata
options_lifesciences = ['Totes les metadades'] + list(lifesciences_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada de ciències de la vida:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_lifesciences,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected life sciences metadata
selected_lifesciences = set()

# Function to save the selected options for life sciences metadata
def save_selection_metadata(change):
    global selected_lifesciences
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_lifesciences = set(list(lifesciences_metadata.keys()))  # Select all life sciences metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_lifesciences.update(
            key for key, value in lifesciences_metadata.items() if value in selected
        )

    print(f"Metadades de ciències de la vida  seleccionades: {list(selected_lifesciences)}")

# Function to clear the selection and reset the life sciences metadata widget
def restart_selection_metadata(button):
    global selected_lifesciences
    metadata_widget.value = []  # Clear the selections in the widget
    selected_lifesciences.clear()  # Clear the global selected set
    print("La selecció de metadades de ciències de la vida  s'ha restablert.")

# Observe changes in the widget selection for life sciences metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for life sciences metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades de ciències de la vida")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# JOURNAL METADATA

# Add an option for selecting all journal metadata
options_journal = ['Totes les metadades'] + list(journal_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada de revista:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_journal,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected journal metadata
selected_journal = set()

# Function to save the selected options for journal metadata
def save_selection_metadata(change):
    global selected_journal
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_journal = set(list(journal_metadata.keys()))  # Select all journal metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_journal.update(
            key for key, value in journal_metadata.items() if value in selected
        )

    print(f"Metadades de revistes seleccionades: {list(selected_journal)}")

# Function to clear the selection and reset the journal metadata widget
def restart_selection_metadata(button):
    global selected_journal
    metadata_widget.value = []  # Clear the selections in the widget
    selected_journal.clear()  # Clear the global selected set
    print("La selecció de metadades de revistes s'ha restablert.")

# Observe changes in the widget selection for journal metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for journal metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades de revistes")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)

########################
# UAB Library METADATA

# Add an option for selecting all customUAB metadata
options_customUAB = ['Totes les metadades'] + list(customUAB_metadata.values())

# Create widgets for instructions and selections
instruction_text_metadata = widgets.HTML(
    value="<b>Trieu alguna metadada de revista:</b>"
)

# Create the widget for multiple selection of metadata
metadata_widget = widgets.SelectMultiple(
    options=options_customUAB,
    value=[],
    description='Metadades:',
    disabled=False
)

# Initialize the global set for selected customUAB metadata
selected_customUAB = set()

# Function to save the selected options for customUAB metadata
def save_selection_metadata(change):
    global selected_customUAB
    selected = set(change['new'])

    if 'Totes les metadades' in selected:
        selected_customUAB = set(list(customUAB_metadata.keys()))  # Select all customUAB metadata elements if 'Totes les metadades' is chosen
    else:
        # Map the selected display names back to the original metadata keys
        selected_customUAB.update(
            key for key, value in customUAB_metadata.items() if value in selected
        )

    print(f"Metadades de revistes seleccionades: {list(selected_customUAB)}")

# Function to clear the selection and reset the customUAB metadata widget
def restart_selection_metadata(button):
    global selected_customUAB
    metadata_widget.value = []  # Clear the selections in the widget
    selected_customUAB.clear()  # Clear the global selected set
    print("La selecció de metadades de revistes s'ha restablert.")

# Observe changes in the widget selection for customUAB metadata
metadata_widget.observe(save_selection_metadata, names='value')

# Create a button to restart the selection for customUAB metadata
restart_button_metadata = widgets.Button(description="Reinicia la selecció de metadades de revistes")
restart_button_metadata.on_click(restart_selection_metadata)

# Display the widgets and buttons with instructions
display(instruction_text_metadata, metadata_widget, restart_button_metadata)



Introduïu el vostre token i premeu enter: 3029e84b-1833-40e5-a0e3-8c9a97cb64bb


HTML(value='<b>Trieu una o més institucions:</b>')

SelectMultiple(description='Institucions:', options=('Totes les institucions', 'UB', 'UAB', 'UPC', 'UPF', 'UdG…

Button(description="Reiniciar la selecció d'institucions", style=ButtonStyle())

HTML(value='<b>Trieu alguna metadada de dates:</b>')

...

Button(description='Reinicia la selecció de metadades de revistes', style=ButtonStyle())

HTML(value='<b>Trieu alguna metadada de revista:</b>')

SelectMultiple(description='Metadades:', options=('Totes les metadades', 'Library reviewing and publishing'), …

Button(description='Reinicia la selecció de metadades de revistes', style=ButtonStyle())

Institucions: ['UAB']
Metadades de dades seleccionades: ['publicationDate']
Metadades de dades seleccionades: ['createTime', 'publicationDate']
Metadades de dades seleccionades: ['createTime', 'publicationDate', 'lastUpdateTime']
Metadades de dades seleccionades: ['createTime', 'releaseTime', 'publicationDate', 'lastUpdateTime']
Metadades de revistes seleccionades: ['reviewLibrary']
Metadades de citació seleccionades: ['publicationCitation']
Metadades de citació seleccionades: ['publicationCitation', 'publicationIDType']
Metadades de citació seleccionades: ['publicationCitation', 'publicationIDNumber', 'publicationIDType']


In [4]:
# @title Un cop escollides les institucions i les metadades, ciqueu el botó &#x25B6; per generar el taulell de metadades
selected_metadata = selected_data | selected_citation | selected_geospatial | selected_social | selected_astronomy | selected_lifesciences | selected_journal | selected_customUAB
from collections import defaultdict
import pandas as pd
metadata_keys_list = []
metadata_values_list = []
list_doi = []
instancia = []
states = []
for element in opcions:
    processor = DatasetProcessor(config, element)
    processor.create_list_datasets(element)
    sigles=element
    for i in processor.get_list_datasets():
        metadata_keys_aux=[]
        metadata_values_aux=[]
        #  Metadata lists:
        data_keys,citation_keys, geo_keys,  social_keys, astronomy_keys, biomedical_keys, journal_keys, customUAB_keys, state = [[] for _ in range(9)]
        data_values,citation_values, geo_values, social_values, astronomy_values, biomedical_values, journal_values, customUAB_values, state = [[] for _ in range(9)]
        data = []
        stateDataset = []
        # Exporting metadata
        export_metadata(config.get_api_url(), config.get_token(), i, data_keys,data_values,citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values, customUAB_keys, customUAB_values, stateDataset)
        # Extracting metadata and arranging it
        extract_metadata(data, data_keys,data_values,citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values, customUAB_keys, customUAB_values)
        # Creating a DataFrame
        df = pd.DataFrame(data, columns=['Metadata', 'Value'])
        metadata_keys_aux = df['Metadata'].tolist()
        metadata_values_aux = df['Value'].tolist()
        metadata_keys_list.append(metadata_keys_aux)
        metadata_values_list.append(metadata_values_aux)
        instancia.append(sigles)
        list_doi.append(i)
        #states.append(stateDataset[0])
def aggregate_metadata(metadata_keys_list, metadata_values_list, list_doi, selected_metadata):
    from collections import defaultdict

    # Initialize dictionaries to store aggregated values
    metadata_values = defaultdict(lambda: defaultdict(set))

    # Aggregate values by DOI
    for i in range(len(metadata_keys_list)):
        doi = list_doi[i]
        for key, value in zip(metadata_keys_list[i], metadata_values_list[i]):
            if key in selected_metadata:
                if isinstance(value, list):
                    metadata_values[key][doi].update(value)
                else:
                    metadata_values[key][doi].add(value)

    # Convert sets to sorted lists
    aggregated_metadata = {field: [''] * len(list_doi) for field in selected_metadata}
    for field in selected_metadata:
        for doi in list_doi:
            values = list(metadata_values[field][doi])
            aggregated_metadata[field][list_doi.index(doi)] = '; '.join(values) if values else ''

    return aggregated_metadata
# Aggregate metadata values
metadata = aggregate_metadata(metadata_keys_list, metadata_values_list, list_doi, selected_metadata)

# Create the data dictionary
data = {
    'DOI': list_doi,
    'Institution': instancia
}

# Add the dynamic metadata fields
for field in selected_metadata:
    data[field] = metadata[field]

# Create the DataFrame
df = pd.DataFrame(data)

# Extract the numeric part of the DOI and convert it to int for sorting
df['DOI_Number'] = df['DOI'].str.extract(r'data(\d+)').astype(int)

# Sort the DataFrame based on the DOI_Number column
df = df.sort_values(by='DOI_Number')

# Format the DOI column as 'https://doi.org/10.34810/dataXXX'
df['DOI'] = 'https://doi.org/10.34810/data' + df['DOI_Number'].astype(str)

# Drop the DOI_Number column (optional)
df = df.drop(columns=['DOI_Number'])

df

S'ha produït un error en llegir les metadades del conjunt de dades: doi:10.34810/data1770


,DOI,Institution,createTime,releaseTime,reviewLibrary,publicationDate,lastUpdateTime
0,https://doi.org/10.34810/data29,UAB,2021-05-07T07:18:22Z,2021-06-16T07:32:56Z,Social Sciences,2021-01-18,2024-07-17T09:21:12Z
205,https://doi.org/10.34810/data585,UAB,2023-09-21T11:03:08Z,2023-09-21T11:05:59Z,,2022-12-16,2023-09-21T11:05:59Z
1,https://doi.org/10.34810/data589,UAB,2023-03-30T10:12:03Z,2023-03-30T10:14:51Z,,2022-12-21,2023-03-30T10:14:51Z
2,https://doi.org/10.34810/data599,UAB,2023-03-30T07:53:28Z,2023-03-30T08:46:14Z,,2023-01-12,2023-03-30T08:46:14Z
3,https://doi.org/10.34810/data601,UAB,2024-03-19T11:20:44Z,2024-03-19T11:22:15Z,,2023-01-20,2024-03-19T11:22:15Z
...,...,...,...,...,...,...,...
195,https://doi.org/10.34810/data2036,UAB,2025-02-06T12:28:23Z,2025-02-10T11:56:29Z,Science and Technology,2025-02-10,2025-02-10T11:56:29Z
202,https://doi.org/10.34810/data2040,UAB,2025-02-12T11:15:33Z,2025-02-13T11:20:16Z,Social Sciences,2025-02-13,2025-02-13T11:20:16Z
203,https://doi.org/10.34810/data2041,UAB,2025-02-12T15:14:02Z,,Social Sciences,,2025-02-13T11:41:31Z
232,https://doi.org/10.34810/data2042,UAB,2025-02-12T16:09:05Z,,Medicine,,2025-02-12T16:09:06Z


In [5]:
# @title Clicar el botó &#x25B6; per guardar les dades en un fitxer excel.
from google.colab import files
# Save the DataFrame to an Excel file
excel_filename = 'estudi_metadades.xlsx'
df.to_excel(excel_filename, index=False)

# Provide download link for the file
files.download(excel_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>